In [ ]:
#  3145tttt/arm_gpt_GPT2_88_12_768

In [36]:
from transformers import AutoTokenizer, GPT2LMHeadModel
from src.util import set_global_seed
from src.eval_promts import clean_text, EVAL_PROMTS
import torch

In [37]:
checkpoint = '3145tttt/arm_gpt_GPT2_88_12_768'

In [53]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = GPT2LMHeadModel.from_pretrained(checkpoint)

device = 'cpu'


In [54]:
def generate_prompt(prompt, max_new_chars=100, temperature=0.1, top_k=100):
    set_global_seed(42)
    cleaned_prompt = clean_text(prompt)
    context = 256
    if len(cleaned_prompt) > context:
        print(f"Warning: Cleaned prompt is too long ({len(cleaned_prompt)} > {context} chars). Truncating.")
        cleaned_prompt = cleaned_prompt[-context:]
    token_ids = tokenizer.encode(cleaned_prompt)
    input_ids = torch.tensor([token_ids], dtype=torch.long, device=device)
    output_ids = model.generate(
        input_ids, max_new_tokens=max_new_chars, temperature=temperature,
        top_k=top_k, do_sample=True, pad_token_id=3
    )
    full_output = tokenizer.decode(output_ids[0].tolist(), skip_special_tokens=True)
    return full_output

In [55]:
for i, prompt in enumerate(EVAL_PROMTS):
    print(f"==========prompt {i+1}==========")
    print(f"prompt: {prompt}")
    generated_text = generate_prompt(prompt)
    print(f"generate: {generated_text}")

==========prompt 1==========
prompt: Ով է գրել Եվգենի Օնեգինին:
generate: ով է գրել եվգենի օնեգինին: այս աշխատությունը հրատարակվել է 1888 թվականին և առաջին անգամ հրատարակվել է 1889 թվականին
"այս աշխատ
==========prompt 2==========
prompt: Պուշկինի անունը
generate: պուշկինի անունը կրող արձանագրությունները հայտնաբերվել են 19 րդ դարում և հայտնաբերվել են միայն 19 րդ դարում
"այս պու
==========prompt 3==========
prompt: Երկիրը արեգակնային համակարգի մոլորակ է, օրինակ
generate: երկիրը արեգակնային համակարգի մոլորակ է, օրինակ արեգակնային համակարգի մոլորակների մոլորակների մոտ"
"այս երկիրը բաժանված է երեք մասի արևմտյան և արևե
==========prompt 4==========
prompt: Լավագույն անիմեն է
generate: լավագույն անիմեն է համարվում ամենահայտնի անիմեներից մեկը
"այս անիմեն համարվում է ամենահայտնի անիմեներից մեկը, որը համա
==========prompt 5==========
prompt: Տիտանիկը հայտնի է նրանով
generate: տիտանիկը հայտնի է նրանով, որ այն առաջացել է արաբական աշխարհում այն ժամանակ, երբ այն առաջացել է արաբական աշխարհում այն առաջ

In [56]:
questions = [
    "Որտեղ է գտնվում Երևանը? Պատասխան:",
    "Տոլստոյը գրել է պատերազմ և խաղաղություն? Պատասխան:",
    "Երևանը Հայաստանի մայրաքաղաքն է? Պատասխան",
    "ով է գրել 'Եվգենի Օնեգին'?: Պատասխան:"
]

In [57]:
for i, prompt in enumerate(questions):
    print(f"==========prompt {i+1}==========")
    print(f"prompt: {prompt}")
    generated_text = generate_prompt(prompt)
    print(f"generate: {generated_text}")

==========prompt 1==========
prompt: Որտեղ է գտնվում Երևանը? Պատասխան:
generate: որտեղ է գտնվում երևանը? պատասխան: այս անգամ արդեն հայտնի է դառնում, որ այն կարող է հանդիսանալ արդեն անհայտ անձի կողմից արդեն անհայտ ա
==========prompt 2==========
prompt: Տոլստոյը գրել է պատերազմ և խաղաղություն? Պատասխան:
generate: տոլստոյը գրել է պատերազմ և խաղաղություն? պատասխան: այս ամենը հանգեցրել է նրան, որ այդ ժամանակ արդեն հայտնի էր որպես արդեն հայտնի արդյունաբերական քաղաք
==========prompt 3==========
prompt: Երևանը Հայաստանի մայրաքաղաքն է? Պատասխան
generate: երևանը հայաստանի մայրաքաղաքն է? պատասխանատու է հայաստանի հանրապետության համար հայաստանի հանրապետության արտակարգ և լիազոր դեսպանի պաշտոնը զբա
==========prompt 4==========
prompt: ով է գրել 'Եվգենի Օնեգին'?: Պատասխան:
generate: ով է գրել 'եվգենի օնեգին'?: պատասխան: այս ամենը հանգեցրեց նրան, որ նա այդ ժամանակ արդեն անցել էր այդ ամենը"
"այս ամենը հանգեցրեց նրան, որ


In [58]:
FEWSHOT = [
    "Պուշկին - Ալեքսանդր. Տոլստոյ - Լև. Գոգոլ - ",
    "Տոլստոյ - Լև. Գոգոլ - Նիկոլայ. Պուշկին - ",
    "Գոգոլ - Նիկոլայ. Պուշկին - Ալեքսանդր. Տոլստոյ - "
]

# Пушкин -> Александр, Толстой - Лев, Гоголь - Николая 
# Николая Гоголь Նիկոլայ Գոգոլ
# Александр Пушкин Ալեքսանդր Պուշկին
# Лев Толстой Լև Տոլստոյ

for i, prompt in enumerate(FEWSHOT):
    print(f"==========prompt {i+1}==========")
    print(f"prompt: {prompt}")
    generated_text = generate_prompt(prompt)
    print(f"generate: {generated_text}")

==========prompt 1==========
prompt: Պուշկին - Ալեքսանդր. Տոլստոյ - Լև. Գոգոլ - 
generate: պուշկին ալեքսանդր. տոլստոյ լև. գոգոլի անվան մրցանակի դափնեկիր (1997)
"արժանացել է ռուսաստանի դաշնության նախագահի կոչման, ռուսաստանի դաշն
==========prompt 2==========
prompt: Տոլստոյ - Լև. Գոգոլ - Նիկոլայ. Պուշկին - 
generate: տոլստոյ լև. գոգոլ նիկոլայ. պուշկինի հետ միասին
"այս ամենի հետ մեկտեղ նա հանդիպում է մի աղջկա, ով այդ ժամանակ ապրում էր այդ տանը"
"այս 
==========prompt 3==========
prompt: Գոգոլ - Նիկոլայ. Պուշկին - Ալեքսանդր. Տոլստոյ - 
generate: գոգոլ նիկոլայ. պուշկին ալեքսանդր. տոլստոյը հայտարարել է, որ այդ ամենը կարող է լինել անհաջողակ և անհաջողակ"
"այս ամենը գրեթե անհնար էր, քանի ո


In [59]:
# :(